In [1]:
# import needed libs
import cv2
import math
import numpy as np
import mediapipe as mp
import pandas as pd
from mio import ResultIO
from paths import *
from google.protobuf.json_format import MessageToDict
import json

## Define constants

In [2]:
# CONFIG
VID_SUF = ".mp4"
S = "/"

# relinquit
DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

# common frame size in the videos
FRAME_WIDTH = 1280
FRAME_HEIGHT = 720

## define mediapipe vars

In [3]:
# prepare mp
mp_hands = mp.solutions.hands
# Import drawing_utils and drawing_styles.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [4]:
def lm_has_side_and_is_at(lm, side): 
    # side: 0 == left, 1 == right
    if lm.multi_handedness is None: 
        return False, 0
    elif len(lm.multi_handedness) == 1: 
        handedness_dict = MessageToDict(lm.multi_handedness[0])
        return handedness_dict["classification"][0]["index"] == side, 0
    elif len(lm.multi_handedness) == 2:
        return True, side
    else: 
        return False, 0

In [5]:
def lm_has_side_and_is_at(lm, side):
    # This is renewed version of lm_has_side_and_is_at, considering the order is not strict
    mh = lm.multi_handedness
    if mh is None: 
        return False, 0
    for i, hand in enumerate(mh):
        handedness_dict = MessageToDict(hand)
        if handedness_dict["classification"][0]["label"] == side:
            return True, i
    return False, 0

In [6]:
def lm_has_side_and_is_at(lm, side):
    # This is a renewed version of lm_has_side_and_is_at, considering the order is not strict
    mh = lm.multi_handedness
    if mh is None: 
        return False, 0
    
    max_score = -1
    max_index = 0
    found = False
    
    for i, hand in enumerate(mh):
        handedness_dict = MessageToDict(hand)
        classification = handedness_dict["classification"][0]
        if classification["label"] == side:
            if classification["score"] > max_score:
                max_score = classification["score"]
                max_index = i
                found = True

    return found, max_index

In [58]:
el = list(mp_hands.HAND_CONNECTIONS)
el = [list(element) for element in el]

In [22]:
def sol2json(d, json_path): 
    with open(json_path, 'w') as fl:
        has, at = lm_has_side_and_is_at(d, 1)
        if has: 
            ml = (MessageToDict(d.multi_hand_landmarks[at])["landmark"]) # 0 is one of the hands, do this first
            my_dict = {str(i): (d['x'], d['y'], d['z']) for i, d in enumerate(ml)}
        else: 
            my_dict = {str(i): (0, 0, 0) for i in range(21)} # default (0, 0, 0) for all nodes
        outdict = {"edges": el, "features": my_dict}
        fl.write(json.dumps(outdict, separators=(',', ':')))

In [23]:
for vd in os.listdir(det_dir): 
    for clip in os.listdir(det_dir + vd + "/"): 
        vid_graph_dir = graph_dir + vd + "/" + clip + "/"
        mk(vid_graph_dir)   # ?
        print(clip)
        for g in os.listdir(det_dir + vd + "/" + clip + "/"): 
            clip_dir = det_dir + vd + "/" + clip + "/"
            if g.endswith(".pkl"): 
                name, ext = g.split(".")
                d = ResultIO.read(os.path.join(clip_dir, g))
                json_path = os.path.join(graph_dir, "{}.json".format(name))
                sol2json(d, json_path)

B_01_041-LOOK_AT-0POR-30
B_01_022-AWKWARD-0N1N-11
B_01_046-MIND-0NU3-36
B_01_029--CITY2-0LUE-19
B_01_036-FLAG-0PEN-25
B_01_033-WARN-12R6-22
B_01_045-PRINT-0NRH-35
B_01_038-LAST-0PO0-27
B_01_043-TRAFFIC-0JL4-32
B_01_039-VIDEOPHONE-12CM-28
B_01_020-SOME-0JG0-10
B_01_015-HEARING_AID-0KL9-5
B_01_023-RELATIONSHIP-15ES-12
B_01_034-NEW_YORK-0V8G-23
B_01_042-LEARN-0MRO-31
B_01_030-TALL_2-16MO-20
B_01_029-CITY-0LUE-18
B_01_032-IMPOSSIBLE-0KC7-21
B_01_022-AWKWARD-0N1N-982
B_01_018-RESPONSIBILITY-135C-8
B_01_040-SHY-0O0L-29
B_01_026-RUDE-0V4N-15
B_01_044--THREE_2-0JG9-34
B_01_024-FAVORITE-0PO0-13
B_01_044-THREE-0JG9-33
B_01_027-HEADACHE-161D-16
B_01_012-PUZZLED-0TCH-3
B_01_035-HOCKEY-0PNI-24
B_01_016-BLIND-0TNI-6
B_01_019-BATH-0R4M-9
B_01_013-SURPRISE-16IQ-4
B_01_047-BODY-13LB-37
B_01_037-FILM-15NR-26
B_01_017-BORED-0O5M-7
B_01_028-HONEST-1001-17
B_01_011-START-15CB-2
B_01_025-HUNT-0OIJ-14


# Test

In [7]:
clip_dir = det_dir + "Cynthia_mini/" + "B_01_016-BLIND-0TNI-6/"

d = ResultIO.read(os.path.join(clip_dir, "B_01_016-BLIND-0TNI-6_000028.pkl"))

d.multi_handedness

[classification {
   index: 1
   score: 0.9994688630104065
   label: "Right"
 },
 classification {
   index: 0
   score: 0.9425473213195801
   label: "Left"
 },
 classification {
   index: 1
   score: 0.9905588030815125
   label: "Right"
 }]

In [102]:
lm_has_side_and_is_at(d, "Left")

(True, 1)